In [677]:
import zipfile, csv, os, io, string, pickle, math
import numpy as np
import matplotlib.pyplot as plt
import urllib.request as urllib
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from numpy.random import randn
import random

In [678]:
df_train= pd.read_csv("mnist_train.csv")
df_test = pd.read_csv("mnist_test.csv")

In [679]:
data_train=df_train.to_numpy()
data_test=df_test.to_numpy()

In [680]:
TEST_RATIO = 0.15
VALIDATION_RATIO = 0.1
MAX_DIM = 1
MAX_LENGTH = 28
NUM_OF_CLASSES = 10
LR = 0.001
LR_DECAY = 0.95
BETA1 = 0.9
BETA2 = 0.999
NUM_OF_EPOCHS = 4
BATCH_SIZE = 100
SAVE_PATH = 'params.pkl'

In [681]:
X_train = data_train[:,1:]
y_train = data_train[:,0]
X_test =  data_test[:,1:]
y_test =  data_test[:,0]

# Some Functions

In [682]:
def function_builder(name, alpha=1, bias=0):
  def func(X):
    if name == 'Relu':
      X[X<=0] = 0
    elif name == 'LeakyRelu':
      X = np.max(alpha * X, X)
    elif name == 'ELU':
      X = alpha*(np.exp(X[X<=0])-1)
    elif name == 'Tanh':
      X = np.tanh(X)
    elif name == 'Linear':
      X = alpha*X + bias
    return X
  return func

In [683]:
def initializeFilter(size, scale = 1.0):
    stddev = scale/np.sqrt(np.prod(size))
    return np.random.normal(loc = 0, scale = stddev, size = size)

def initializeWeight(size):
    return np.random.standard_normal(size=size) * 0.01

def initializeParams ():
  m = [None]*len(Network)
  v = [None]*len(Network)
  mb = [None]*len(Network)
  vb = [None]*len(Network)

  for i, layer in enumerate(Network):
    if layer[0] == 'maxpool' or layer[0] == 'flatten':
      continue
    m[i] = np.zeros(layer[1].shape)
    v[i] = np.zeros(layer[1].shape)
    mb[i] = np.zeros(layer[2].shape)
    vb[i] = np.zeros(layer[2].shape)

  return [m, v, mb, vb]

In [684]:
def categoricalCrossEntropy(probs, label):
    return -np.sum(label * np.log(probs)) 

In [685]:
def nanargmax(arr):
    idx = np.nanargmax(arr)
    idxs = np.unravel_index(idx, arr.shape)
    return idxs

In [686]:
def calc_accuracy (X_test,Y_true):
  
  Y_pred = np.empty((np.shape(Y_true)))
  Y_probs = np.empty((np.shape(Y_true)))
  for i in range(len(X_test)):
    Y_pred[i], Y_probs[i] = predict(X_test[i])
  Y_pred = np.array(Y_pred).astype(int)
  return np.sum((Y_pred==Y_true).astype(int))/len(Y_true)*100, Y_pred

# Layer Builders

In [687]:
def convolution(data, weights, bias, stride):
    filter_x, filter_y, filter_depth, filter_num = np.shape(weights)
    data_x, data_y, data_depth = np.shape(data)
    result = np.zeros((data_x, data_y, filter_num))
    pad_size_x = filter_x // 2
    pad_size_y = filter_y // 2
    padded_data = np.pad(data, ((pad_size_x, pad_size_x), (pad_size_y, pad_size_y), (0, 0)), 'constant', constant_values=0)
    for n in range(filter_num):
        kernel_dy = pad_size_y
        result_y = 0
        while kernel_dy < data_y:
            kernel_dx = pad_size_x
            result_x  = 0
            while kernel_dx < data_x:
                result[result_x, result_y, n] = np.sum(np.multiply(weights[:, :, :, n], padded_data[kernel_dx:kernel_dx+filter_x, kernel_dy:kernel_dy+filter_y, :])) + bias[n]
                kernel_dx += stride
                result_x += 1
            kernel_dy += stride
            result_y += 1
    return result

In [688]:
def maxpool(data, pool_f): #pool_f is tuple (2, 1), pool_stride
    (data_x, data_y, data_depth) = np.shape(data)
    dim_x = math.ceil(data_x / pool_f[0])
    dim_y = math.ceil(data_y / pool_f[1])
    downsampled = np.zeros((dim_x, dim_y, data_depth))
    pad_size_x = pool_f[0] - data_x % pool_f[0]
    pad_size_y = pool_f[1] - data_y % pool_f[1]
    padded_data = np.pad(data, ((0, pad_size_x), (0, pad_size_y), (0, 0)), 'constant', constant_values=0)
    for i in range(data_depth):
        kernel_dy = 0
        result_y = 0
        while kernel_dy + pool_f[1] < np.shape(padded_data)[1]:
            kernel_dx   = 0
            result_x = 0
            while kernel_dx + pool_f[0] < np.shape(padded_data)[0]:
                downsampled[result_x, result_y, i] = np.max(padded_data[kernel_dx:kernel_dx+pool_f[0], kernel_dy:kernel_dy+pool_f[1], i])
                kernel_dx += pool_f[0]
                result_x += 1
            kernel_dy += pool_f[1]
            result_y += 1
    return downsampled

In [689]:
def convolutionBackward(dconv_prev, conv_in, filt, s):
   
    (f_dim_x, f_dim_y, f_depth, num_f) = filt.shape
    (orig_dim_x,orig_dim_y , _) = conv_in.shape
   
    dout = np.zeros(conv_in.shape) 
    dfilt = np.zeros(filt.shape)
    dbias = np.zeros((num_f,1))

    for curr_f in range(num_f):
      
        curr_y = out_y = 0
        while curr_y + f_dim_y <= orig_dim_y:
            curr_x = out_x = 0
            while curr_x + f_dim_x <= orig_dim_x:
                dfilt[: , : , : ,curr_f] += dconv_prev[out_x, out_y,curr_f] * conv_in[curr_x:curr_x + f_dim_x, curr_y:curr_y + f_dim_y,:]  ###########################         
                dout[curr_x:curr_x + f_dim_x, curr_y:curr_y + f_dim_y,:] += dconv_prev[out_x, out_y,curr_f] * filt[: , : , : ,curr_f] 
                curr_x += s
                out_x += 1
            curr_y += s
            out_y += 1
       
        dbias[curr_f] = np.sum(dconv_prev[: , : ,curr_f])
    
    return dout, dfilt, dbias

In [690]:
def maxpoolBackward(dpool, orig, pool_f):
   
    (orig_dim_x, orig_dim_y, orig_depth) = orig.shape
    
    dout = np.zeros(orig.shape)
    
    for curr_c in range(orig_depth):
        curr_y = out_y = 0
        while curr_y + pool_f[1] <= orig_dim_y:
            curr_x = out_x = 0
            while curr_x + pool_f[0] <= orig_dim_x:
                
                (a, b) = nanargmax(orig[curr_x:curr_x+ pool_f[0] , curr_y:curr_y + pool_f[1], curr_c])
                dout[ curr_x+a, curr_y+b, curr_c] = dpool[out_x, out_y, curr_c]
                
                curr_x += pool_f[0]
                out_x += 1
            curr_y += pool_f[1]
            out_y += 1
        
    return dout

In [691]:
def softmax(raw_preds):
    out = np.exp(raw_preds) 
    return out/np.sum(out) 

# Design Network

In [705]:
# conv, maxpool, fullyconn, flatten, softmax
Network = []
Network.append(['conv', initializeFilter((5, 5, 1, 8)), np.zeros((8,1)), 'Relu']) ##########
Network.append(['conv', initializeFilter((5, 5, 8, 8)), np.zeros((8,1)), 'Relu'])
Network.append(['maxpool', (2, 2)])


#Network.append(['conv', initializeFilter((5, 5, 8, 16)), np.zeros((16,1)), 'Relu']) ##########
#Network.append(['conv', initializeFilter((5, 5, 16, 16)), np.zeros((16,1)), 'Relu'])
#Network.append(['maxpool', (2, 2)])

#Network.append(['conv', initializeFilter((5, 5, 16, 32)), np.zeros((32,1)), 'Relu']) ##########
#Network.append(['conv', initializeFilter((5, 5, 32, 32)), np.zeros((32,1)), 'Relu'])
#Network.append(['maxpool', (2, 2)])

#Network.append(['conv', initializeFilter((3, 1, 32, 32)), np.zeros((32,1)), 'Relu'])
Network.append(['flatten',])

Network.append(['fullyconn', initializeWeight((200, 1568)), np.zeros((200,1)), 'Relu'])
Network.append(['fullyconn', initializeWeight((10, 200)), np.zeros((10,1)), 'Softmax'])



In [706]:
data_dim_x = MAX_LENGTH
data_dim_y = 1
data_depth = MAX_DIM
f_dim1 = 3
f_dim2 = 1
num_filt1 = 64
num_filt2 = 64
num_neurons = 100
flatten_size = 1600

# Implementing Network

In [707]:
def CNN_forward (data):
    results = [None]*(len(Network)+1)
    results[0] = data
    for i, layer in enumerate(Network):
      if (layer[0] == 'conv'):
        results[i+1] = convolution(results[i], layer[1], layer[2], 1)
        results[i+1] = function_builder(layer[3])(results[i+1])
      elif (layer[0] == 'maxpool'):
        results[i+1] = maxpool(results[i], layer[1])
      elif (layer[0] == 'flatten'):
        dim1, dim2, dim3 = np.shape(results[i])
        results[i+1] = results[i].reshape((dim1*dim2*dim3, 1))
      elif (layer[0] == 'fullyconn'):
        results[i+1] = layer[1].dot(results[i]) + layer[2]
        if layer[3] == 'Softmax':
          results[i+1] = softmax(results[i+1])
          break
        results[i+1] = function_builder(layer[3])(results[i+1])
    return results

In [708]:
def CNN_backward (results, label):
    dNetwork = [[None, None]]*len(Network)
    dresults = [None]*(len(Network)+1)
    dresults[-1] = results[-1] - label
    for i, layer in reversed(list(enumerate(Network))):
      if (layer[0] == 'conv'):
        dresults[i], df, db = convolutionBackward(dresults[i+1], results[i], layer[1], 1)
        dNetwork[i] = [df, db]
        if i != 0 and Network[i-1][0] == 'conv':
          dresults[i][results[i]<=0] = 0
      elif (layer[0] == 'maxpool'):
        dresults[i] = maxpoolBackward(dresults[i+1], results[i], layer[1])
        dresults[i][results[i]<=0] = 0
      elif (layer[0] == 'flatten'):
        dresults[i] = dresults[i+1].reshape(np.shape(results[i]))
      elif (layer[0] == 'fullyconn'):
        dw = dresults[i+1].dot(results[i].T)
        db = np.sum(dresults[i+1], axis=1).reshape(layer[2].shape)
        dNetwork[i] = [dw, db]
        dresults[i] = layer[1].T.dot(dresults[i+1])
        if Network[i-1][0] == 'flatten':
          continue
        dresults[i][results[i]<=0] = 0
    return dNetwork

In [709]:
def CNN (data, label):
    
    results = CNN_forward (data)
    dNetwork = CNN_backward(results, label)

    loss = categoricalCrossEntropy(results[-1], label)
    grads = dNetwork

    return grads, loss

# Solver

In [710]:
def momentumGD(dNetwork, lr, params):
    [m, v, mb, vb] = params
    for i, layer in enumerate(Network):

      if layer[0] == 'maxpool' or layer[0] == 'flatten':
        continue
      v[i] = GAMMA*v[i] + lr*(dNetwork[i][0]/BATCH_SIZE)
      layer[1] -= v[i]

      vb[i] = GAMMA*vb[i] + lr*(dNetwork[i][1]/BATCH_SIZE)
      layer[2] -= vb[i]

    return [m, v, mb, vb]

In [711]:
def adamGD (dNetwork, lr, params):
    [m, v, mb, vb] = params
    for i, layer in enumerate(Network):

      if layer[0] == 'maxpool' or layer[0] == 'flatten':
        continue
      m[i] = BETA1*m[i] + (1-BETA1)*dNetwork[i][0]/BATCH_SIZE 
      v[i] = BETA2*v[i] + (1-BETA2)*(dNetwork[i][0]/BATCH_SIZE)**2 
      m_hat = m[i]/(1-BETA1)
      v_hat = v[i]/(1-BETA2)
      layer[1] -= lr * m_hat/(np.sqrt(v_hat)+1e-7)

      mb[i] = BETA1*mb[i] + (1-BETA1)*dNetwork[i][1]/BATCH_SIZE
      vb[i] = BETA2*vb[i] + (1-BETA2)*(dNetwork[i][1]/BATCH_SIZE)**2
      mb_hat = mb[i]/(1-BETA1)
      vb_hat = vb[i]/(1-BETA2)
      layer[2] -= lr * mb_hat/(np.sqrt(vb_hat)+1e-7)

    return [m, v, mb, vb]

In [712]:
def solver(batch, lr, params, cost, optimizer='ADAM'):
    
    X = batch[:,0:-1] 
    Y = batch[:,-1] 

    X = X.reshape(len(batch),MAX_LENGTH,MAX_LENGTH,MAX_DIM)   
   
    cost_ = 0
    
    dNetwork = [[0, 0]]*len(Network)
    
    for i in range(len(X)):
        
        x = X[i]
        y = np.eye(NUM_OF_CLASSES)[int(Y[i])].reshape(NUM_OF_CLASSES, 1) 

        grads, loss = CNN(x, y)
        for j, grad in enumerate(grads):
          if grad[0] is None:
            continue
          if i == 0:
            dNetwork[j] = [np.zeros(grad[0].shape), np.zeros(grad[1].shape)]
          dNetwork[j][0] += grad[0]
          dNetwork[j][1] += grad[1]
        cost_+= loss

    if optimizer == 'ADAM':
      new_params = adamGD(dNetwork, lr, params)
    elif optimizer == 'MOMENTUM':
      new_params = momentumGD(dNetwork, lr, params)

    cost_ = cost_/BATCH_SIZE
    cost.append(cost_)

    return new_params, cost

# Predictor

In [713]:
def predict(data):
    results = CNN_forward(data)
    return np.argmax(results[-1]), np.max(results[-1])

In [714]:
params = initializeParams()

m =5000
X = X_train[0:m,:]
len(X)
y_dash = y_train[0:m].reshape(m,1)
mean=int(np.mean(X))
std= int(np.std(X))
X=(X-mean)/std
trainData = np.hstack((X,y_dash))
cost = []
print("LR:"+str(LR)+", Batch Size:"+str(BATCH_SIZE))
for epoch in range(NUM_OF_EPOCHS):
    np.random.shuffle(trainData)
    batches = np.array([trainData[k:k + BATCH_SIZE] for k in range(0, trainData.shape[0], BATCH_SIZE)])
    t = tqdm(batches)
    for x,batch in enumerate(t):
      
        params, cost = solver(batch, LR*(LR_DECAY**x), params, cost, 'ADAM')
        t.set_description("Cost: %.6f" % (cost[-1]))
with open(SAVE_PATH, 'wb') as file:
    pickle.dump(Network, file)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

LR:0.001, Batch Size:100


Cost: 0.458555: 100%|██████████████████████████████████████████████████████████████████| 10/10 [05:06<00:00, 30.68s/it]


# Loading Parameters from Directory

In [ ]:
Network = pickle.load(open('pramas.plk', 'rb'))

# Test 

In [716]:
m =1000
X = X_train[0:m,:]
len(X)
y_dash = y_train[0:m].reshape(m,1)
mean=int(np.mean(X))
std= int(np.std(X))
#################################################
m =200
X = X_test[0:m,:]
y_dash = y_test[0:m]
X=(X-mean)/std
X = X.reshape(len(X),28,28,1)
acc, Y_pred = calc_accuracy(X,y_dash)
print("accuracy is equal to:",acc)

accuracy is equal to: 89.0


# Image Generation

In [717]:
def CNN_backward (results, label):
    dNetwork = [[None, None]]*len(Network)
    dresults = [None]*(len(Network)+1)
    dresults[-1] = results[-1] - label
    for i, layer in reversed(list(enumerate(Network))):
      if (layer[0] == 'conv'):
        dresults[i], df, db = convolutionBackward(dresults[i+1], results[i], layer[1], 1)
        dNetwork[i] = [df, db]
        if i != 0 and Network[i-1][0] == 'conv':
          dresults[i][results[i]<=0] = 0
      elif (layer[0] == 'maxpool'):
        dresults[i] = maxpoolBackward(dresults[i+1], results[i], layer[1])
        dresults[i][results[i]<=0] = 0
      elif (layer[0] == 'flatten'):
        dresults[i] = dresults[i+1].reshape(np.shape(results[i]))
      elif (layer[0] == 'fullyconn'):
        dw = dresults[i+1].dot(results[i].T)
        db = np.sum(dresults[i+1], axis=1).reshape(layer[2].shape)
        dNetwork[i] = [dw, db]
        dresults[i] = layer[1].T.dot(dresults[i+1])
        if Network[i-1][0] == 'flatten':
          continue
        dresults[i][results[i]<=0] = 0
    return dNetwork,dresults[0]

In [718]:
def CNN (data, label):
    
    results = CNN_forward (data)
    dNetwork,dresults = CNN_backward(results, label)

    loss = categoricalCrossEntropy(results[-1], label)
    grads = dNetwork

    return dresults, loss

# Noise 

In [719]:
def generate_flat_points(latent_dim, n_samples):
    
    x_input=255 * np.random.rand(28, 28, 1)
    return x_input

In [723]:
def IMAGE_GENERATOR(label,beta1,beta2,lr):
    our_noise=255 * np.random.rand(28, 28, 1)

    image=np.zeros((28, 28, 1))

    image=image-mean
    image=image/std

    v1 = np.zeros(image.shape)
    s1 = np.zeros(image.shape)

    for i in range(200):
    
        y = np.eye(10)[int(label)].reshape(10, 1) 
        dresults,loss=CNN(image,y)
        print(dresults.shape)
        v1 = beta1*v1 + (1-beta1)*dresults
        s1 = beta2*s1 + (1-beta2)*(dresults)**2 
        v1_hat = v1/(1-beta1)
        s1_hat = s1/(1-beta2)
        image-= lr * v1_hat/(np.sqrt(s1_hat)+1e-7)
        print(loss)
    
    plt.imshow(image)

# Image generator

In [ ]:
beta1 = 0.9
beta2 = 0.999
lr = 0.001
label=3
IMAGE_GENERATOR(label,beta1,beta2,lr)
